## Selecting Rows with Core or ORM

For both Core and ORM, the `select()` function generates a `Select construct` which is used for all `SELECT` queries. Passed to methods like _`Connection.execute()` in Core_ and _`Session.execute()` in ORM_, a `SELECT statement` is emitted in the _current transaction_ and the result rows available via the returned `Result` object.

#### Initialize tables with some data

In [1]:
from sqlalchemy import (
    MetaData, Table, Column, Integer, String,
    ForeignKey, create_engine, insert, select, bindparam,
)

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj = MetaData()

In [3]:
user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

2022-09-22 09:47:34,566 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 09:47:34,568 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-22 09:47:34,569 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 09:47:34,570 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-22 09:47:34,571 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 09:47:34,573 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-22 09:47:34,574 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 09:47:34,576 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-22 09:47:34,578 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 09:47:34,580 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-22 09:47:34,581 INFO sqlalchemy.engine.Engine [no key 0.00132s] ()
2022-09-22 09:47:34,585 INFO sqlalchemy.engine.Engine 
C

In [4]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-09-22 09:47:35,520 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 09:47:35,522 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-22 09:47:35,523 INFO sqlalchemy.engine.Engine [generated in 0.00238s] ('spongebob', 'Spongebob Squarepants')
2022-09-22 09:47:35,528 INFO sqlalchemy.engine.Engine COMMIT
2022-09-22 09:47:35,531 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 09:47:35,532 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-22 09:47:35,534 INFO sqlalchemy.engine.Engine [generated in 0.00309s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-09-22 09:47:35,536 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam("username")).
    scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()

2022-09-22 09:47:36,889 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 09:47:36,891 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-09-22 09:47:36,893 INFO sqlalchemy.engine.Engine [generated in 0.00392s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-09-22 09:47:36,896 INFO sqlalchemy.engine.Engine COMMIT
